######The latest version of this IPython notebook is available at [http://github.com/jckantor/CBE20255](http://github.com/jckantor/CBE20255)  for noncommercial use under terms of the [Creative Commons Attribution Noncommericial ShareAlike License](http://creativecommons.org/licenses/by-nc-sa/4.0/).

J.C. Kantor (Kantor.1@nd.edu)

# Ethylene Oxide Flowsheet

This [IPython notebook](http://ipython.org/notebook.html) demonstrates the formulation and solution of material balances for an ethylene oxide flowsheet using the [symbolic algebra package Sympy](http://sympy.org/en/index.html).

## Introduction

While material balances for chemical processes are readily expressed as linear equations, extracting the matrix of coefficients for the linear equations can be tedious and error-prone. Fortunately, symbolic calculations can be used to solve material balance problems typical of introductory chemical engineering courses. This is demonstrated below using an example from the textbook [*
Introduction to Chemical Processes: Principles, Analysis, and Synthesis*](http://highered.mcgraw-hill.com/sites/0072849606/) by Regina Murphy.

## Ethylene Oxide

<p><a href="http://commons.wikimedia.org/wiki/File:TASNEE_001.jpg#/media/File:TASNEE_001.jpg"><img src="http://upload.wikimedia.org/wikipedia/commons/thumb/d/d2/TASNEE_001.jpg/1200px-TASNEE_001.jpg" alt="TASNEE 001.jpg"></a><br>"<a href="http://commons.wikimedia.org/wiki/File:TASNEE_001.jpg#/media/File:TASNEE_001.jpg">TASNEE 001</a>" by <a href="//commons.wikimedia.org/w/index.php?title=User:Secl&amp;action=edit&amp;redlink=1" class="new" title="User:Secl (page does not exist)">Secl</a> - <span class="int-own-work" lang="en">Own work</span>. Licensed under <a href="http://creativecommons.org/licenses/by/3.0" title="Creative Commons Attribution 3.0">CC BY 3.0</a> via <a href="//commons.wikimedia.org/wiki/">Wikimedia Commons</a>.</p>

## Problem Statement (Murphy 2005, Example 3.19)

The problem is to analyze the capability of an existing process for the production of [ethylene oxide](http://www.americanchemistry.com/ProductsTechnology/Ethylene-Oxide/PDF-Ethylene-Oxide-An-Essential-Raw-Material-for-Many-Important-Products-chart.pdf) by the reaction of ethylene and oxygen

$$2\,C_2H_4 + O_2 \longrightarrow 2\,C_2H_4O$$

The target production is 1.7 &#215; 10<sup>6</sup> kgmol/year of an ethylene oxide product with 98 mol% purity. The feedrate of ethylene is specified as 196 kgmol/hr, and of oxygen is specified as 84.5 kgmol/hr. The reactor has a nominal single pass conversion of 6% ethylene. The separator recovers 98% of the ethylene and 99.5% of the oxygen and 98% of the ethylene for recycle, and 97% of the ethylene oxide for the product stream.  

<img src="https://raw.github.com/jckantor/CBE20255/master/notebooks/images/ethylene_oxide_flowsheet.png" style="width:500px">

The problem is to find the nominal product purity and production rates, and to examine the sensitivity of product purity and production to the equipment specifications.

## Solution

[Sympy](http://sympy.org/en/index.html) is a library written in pure python for symbolic algebra. The solution strategy is to represent the stream variables and reaction extents as symbolic variables, then express the material balances and process specifications as symbolic equations, and finally to use the Sympy solver to find a nominal solution and to perform parametric analyses.

In [1]:
import sympy as sym
sym.init_printing()

from IPython.display import display

### Variables

#### Stream Variables

The stream variables are systematically created using the Sympy `var` function, and added to the list `streams`.

In [2]:
stream_table = []
stream_table.append(sym.var('n_E1 n_O1 n_EO1'))   # Stream 1
stream_table.append(sym.var('n_E2 n_O2 n_EO2'))   # Stream 2
stream_table.append(sym.var('n_E3 n_O3 n_EO3'))   # Stream 3
stream_table.append(sym.var('n_E4 n_O4 n_EO4'))   # Stream 4
stream_table.append(sym.var('n_E5 n_O5 n_EO5'))   # Stream 5

display(stream_table)

[(n_E1, n_O1, n_EO1),
 (n_E2, n_O2, n_EO2),
 (n_E3, n_O3, n_EO3),
 (n_E4, n_O4, n_EO4),
 (n_E5, n_O5, n_EO5)]

#### Extent of Reaction

This problem includes only a single reaction in a single reactor. A corresponding extent of reaction variable is created, and a master list of all process variables is created.

In [3]:
extents = [sym.var('X')]
display(extents)

[X]

Create list of all variables

In [4]:
variables = []
for x in extents:
    variables.append(x)
for s in stream_table:
    for v in s:
        variables.append(v)

display(variables)

[X,
 n_E1,
 n_O1,
 n_EO1,
 n_E2,
 n_O2,
 n_EO2,
 n_E3,
 n_O3,
 n_EO3,
 n_E4,
 n_O4,
 n_EO4,
 n_E5,
 n_O5,
 n_EO5]

### Equations

#### Material Balances

Material balances are written for each process unit using the symbolic stream and extent of reaction variables created above. Each material balance is expressed as the net rate of accumulation that will be set to zero to specify steady-state operation. The material balances are gathered into a list for each process unit, then the lists are concantenated to create a list of all material balances.

In [5]:
mixer = [
    sym.Eq(0, n_E1 + n_E5 - n_E2),
    sym.Eq(0, n_O1 + n_O5 - n_O2),
    sym.Eq(0, n_EO5 - n_EO2)]

reactor = [
    sym.Eq(0, n_E2 - n_E3 - 2*X),
    sym.Eq(0, n_O2 - n_O3 - X),
    sym.Eq(0, n_EO2 - n_EO3 + 2*X)]

separator = [
    sym.Eq(0, n_E3 - n_E4 - n_E5),
    sym.Eq(0, n_O3 - n_O4 - n_O5),
    sym.Eq(0, n_EO3 - n_EO4 - n_EO5)]

material_balances = mixer + reactor + separator
for eqn in material_balances:
    display(eqn)

0 == n_E1 - n_E2 + n_E5

0 == n_O1 - n_O2 + n_O5

0 == -n_EO2 + n_EO5

0 == -2*X + n_E2 - n_E3

0 == -X + n_O2 - n_O3

0 == 2*X + n_EO2 - n_EO3

0 == n_E3 - n_E4 - n_E5

0 == n_O3 - n_O4 - n_O5

0 == n_EO3 - n_EO4 - n_EO5

#### Specifications

Process specifications are written as equalities using the Sympy `Eq` function.

In [6]:
feed_spec = [
    sym.Eq(n_E1, 196.0),
    sym.Eq(n_O1, 84.5),
    sym.Eq(n_EO1, 0.0)]

reactor_spec = [
    sym.Eq(n_E2 - n_E3, 0.06*E2)]

separator_spec = [
    sym.Eq(n_E5, 0.98*E3),
    sym.Eq(n_O5, 0.995*O3),
    sym.Eq(n_EO4, 0.97*EO3)]

specifications = feed_spec + reactor_spec + separator_spec
for eqn in specifications:
    display(eqn)

NameError: name 'E2' is not defined

### Degree of Freedom Analysis

A simple degree of freedom analysis is to compare the number of variables to the number of equations.

In [ ]:
print "\n%d Variables = %d Stream Variables + %d Extents of Reaction " \
    % (len(variables),len(stream_variables),len(extents))

print "\n%d Stream Variables\n" % len(stream_variables)
for s in streams:
    for v in s:
        print "Variable: ", v.name

print "\n%d Extents of Reaction\n" % len(extents)
for v in extents:
    print "Extent: ", v.name


In [ ]:
equations = material_balances + specifications
print "\n%d Equations = %d Material Balances + %d Specifications" \
    % (len(equations),len(material_balances),len(specifications))

print "\n%d Material Balances\n" % len(material_balances)
for mb in material_balances:
    print mb

print "\n%d Specifications\n" % len(specifications)
for spec in specifications:
    print spec

### Solution

In [ ]:
soln = sym.solve(material_balances + specifications)
display(soln)

In [ ]:
# display solution for each variable, rounded to 1 decimal place
for v in variables:
    display(sym.Eq(v,round(soln[v],1)))

#### Production and Purity

In [ ]:
purity = soln[EO4]/(soln[EO4]+soln[E4]+soln[O4])
production = 24*350*(soln[EO4] + soln[E4] + soln[O4])/1000000

print "Annual Production %4.2f million kgmol/year at %5.3f purity." \
    % (production,purity)

#### Discussion Questions

* Do these numbers surprise you?  Why is the recycle rate so high?
* The purity specification is not met. Why?

### Parametric Analysis: Fractional Conversion of Ethylene

The problem asked for an analysis of the sensitivity of the problem results to changes in unit performance. This is implemented by restating the specifications where a key parameter is replaced by a symbolic variable, and the process the performance plotted as a function of the parameter.

In [ ]:
feed_spec = [
    sym.Eq(E1, 196.0),
    sym.Eq(O1, 84.5),
    sym.Eq(EO1, 0.0)]

fconv = sym.symbols('fconv')
reactor_spec = [
    sym.Eq(E2 - E3, fconv*E2)]

separator_spec = [
    sym.Eq(E5, 0.98*E3),
    sym.Eq(O5, 0.995*O3),
    sym.Eq(EO4, 0.97*EO3)]

specifications = feed_spec + reactor_spec + separator_spec
for s in specifications:
    display(s)

Recycle calculations are introduce a strong dependence of flow rates on parameters such as fraction conversion in the reactor and fractional recovery in separation units. To see this, here we solve for the flowrate of E<sub>2</sub> as a function of fractional conversion of ethylene in the reactor.

From the material balances

E<sub>2</sub> = E<sub>1</sub> + E<sub>5</sub><br>
E<sub>3</sub> = (1-f<sub>conv</sub>) E<sub>2</sub><br>
E<sub>5</sub> = 0.98 E<sub>3</sub><br>

Take a moment and solve these by hand.

In [ ]:
soln = solve(material_balances + specifications)[0]

display(Eq(E2,soln[E2]))
plot(soln[E2],(fconv,0,1),xlabel='fconv',ylabel='kg-mol/hour',title='E2 flowrate');

In [ ]:
for v in variables:
    display(Eq(v, soln[v]))

In [ ]:
purity = soln[EO4]/(soln[EO4]+soln[E4]+soln[O4])

display(simplify(purity))
plot(purity,(fconv,0,.2),xlabel='fconv',ylabel='fraction EO',title='Purity');

In [ ]:
production = 24*350*(soln[EO4] + soln[E4] + soln[O4])/1000000

display(simplify(production))
plot(production,(fconv,0,0.2),xlabel='fconv',ylabel='kg-mol/year',title='Production');

### Parametric Analysis: Fractional Recovery of Ethylene Oxide to Product

Will improving the recovery of Ethylene Oxide to the product stream improve the product purity? Make a prediction, then check against the results of the following calculations.

In [ ]:
feed_spec = [
    Eq(E1,196.0),
    Eq(O1, 84.5)]

reactor_spec = [
    Eq(E2 - E3, 0.06*E2)]

frcvr = symbols('frcvr')
separator_spec = [
    Eq(E5, 0.98*E3),
    Eq(O5, 0.995*O3),
    Eq(EO4, frcvr*EO3)]

specifications = feed_spec + reactor_spec + separator_spec
for s in specifications:
    display(s)

In [ ]:
soln = solve(material_balances + specifications)[0]

purity = soln[EO4]/(soln[EO4]+soln[E4]+soln[O4])

display(purity)
plot(purity,(frcvr,0.9,1.00),xlabel='frcvr',ylabel='fraction EO',title='Purity');

In [ ]:
production = 24*350*(soln[EO4] + soln[E4] + soln[O4])/1000000
display(production)
plot(production,(frcvr,0.9,1.00),xlabel='frcvr',ylabel='kg-mol/year',title='Production');

#### Discussion Questions

* Is this what you expected?
* Why doesn't the product purity or production depend on the fractional recovery of ethylene oxide in the separator?

### Parametric Analysis: Fractional Recovery of Ethylene to Recycle

Increasing the fraction of ethylene recovered for recycle should improve product purity. Let's see what happens when we raise it.

In [ ]:
feed_spec = [
    Eq(E1,196.0),
    Eq(O1, 84.5)]

reactor_spec = [
    Eq(E2 - E3, 0.06*E2)]

frcvr = symbols('frcvr')
separator_spec = [
    Eq(E5, frcvr*E3),
    Eq(O5, 0.995*O3),
    Eq(EO4, 0.97*EO3)]

specifications = feed_spec + reactor_spec + separator_spec
for s in specifications:
    display(s)

In [ ]:
soln = solve(material_balances + specifications)[0]

purity = soln[EO4]/(soln[EO4]+soln[E4]+soln[O4])
display(simplify(purity))
plot(purity,(frcvr,0.9,1),xlabel='frcvr',ylabel='fraction EO',title='Purity');

In [ ]:
production = 24*350*(soln[EO4] + soln[E4] + soln[O4])/1000000
display(simplify(production))
plot(production,(frcvr,0.9,1.00),xlabel='frcvr',ylabel='kg-mol/year',title='Production');

#### Discussion Questions

* Did this behave as you expected?
* Increasing the ethylene recovery to 1.00 leads to a product purity greater than 1.0. Obviously that's not possible. What's going wrong?
* What is the maximum possible recovery of ethylene for recycle?

#### Parametric Analysis: Change the Oxygen Feed Rate

As we've discovered, the oxygen feedrate is not sufficient to consume all of the ethylene. Let's explore what happens if we change the oxygen feedrate.

In [ ]:
oxyfeed = symbols('oxyfeed')
feed_spec = [
    Eq(E1,196.0),
    Eq(O1,oxyfeed)]

reactor_spec = [
    Eq(E2 - E3, 0.06*E2)]

separator_spec = [
    Eq(E5, 0.98*E3),
    Eq(O5, 0.995*O3),
    Eq(EO4, 0.97*EO3)]

specifications = feed_spec + reactor_spec + separator_spec
for s in specifications:
    display(s)

In [ ]:
soln = solve(material_balances + specifications)

purity = soln[EO4]/(soln[EO4]+soln[E4]+soln[O4])
display(simplify(purity))
plot(purity,(oxyfeed,74.5,196/2),xlabel='O1',ylabel='fraction EO',title='Purity');

In [ ]:
production = 24*350*(soln[EO4] + soln[E4] + soln[O4])/1000000
display(simplify(production))
plot(production,(oxyfeed,74.5,196/2),xlabel='O1',ylabel='kg-mol/year',title='Production');

####     Discussion Questions

* Did this behave as you expected?
* Why did the product purity decrease as the oxygen feedrate was increased?

## Conclusions

* The nominal process specifications yield a product purity of 72.5 mol% and a production of 1.73 million kgmol/year. The product purity falls significantly short of the desired purity of 98 mol%.
* Product purity can be increased by increasing the single-pass reactor conversion, increasing the recovery of ethylene to recycle, and decreasing oxygen feed. Individual, however, these changes are not sufficient to meet to the desired purity.
* Can you find specifications that will meet the 98% product purity specification?